## Lexicase Selection 词典序选择 注意事项

对于Lexicase Selection，适应度评估需要更改为返回多个误差组成的向量，而不是均方误差（MSE）。这样，Lexicase Selection才能独立考虑每个个体在每个测试样本上的表现，从而提高选择的多样性。

In [1]:
import numpy as np
import math
import operator

from deap import base, creator, tools, gp


# 符号回归
def evalSymbReg(individual, pset):
    # 编译GP树为函数
    func = gp.compile(expr=individual, pset=pset)
    
    # 使用numpy创建一个向量
    x = np.linspace(-10, 10, 100) 
    
    return tuple((func(x) - x**2)**2)


# 创建个体和适应度函数，适应度数组大小与数据量相同
creator.create("FitnessMin", base.Fitness, weights=(-1.0,) * 100)  # 假设我们有20个数据点
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

### 遗传算子
选择算子需要改成Lexicase Selection，其他不需要改变。对于回归问题，需要使用AutomaticEpsilonLexicase。而对于分类问题，则使用Lexicase即可。

In [2]:
import random

# 定义函数集合和终端集合
pset = gp.PrimitiveSet("MAIN", arity=1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(operator.neg, 1)
pset.addEphemeralConstant("rand101", lambda: random.randint(-1, 1))
pset.renameArguments(ARG0='x')

# 定义遗传编程操作
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("evaluate", evalSymbReg, pset=pset)
# 使用AutomaticEpsilonLexicase选择算子
toolbox.register("select", tools.selAutomaticEpsilonLexicase)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr, pset=pset)

/home/glk/project/anaconda3/lib/python3.11/site-packages/deap/gp.py:257: RuntimeWarning: Ephemeral rand101 function cannot be pickled because its generating function is a lambda function. Use functools.partial instead.
  warnings.warn("Ephemeral {name} function cannot be "


特别解释一下AutomaticEpsilonLexicase选择算子：
1. 这是Lexicase Selection的一个变体
2. 与普通的Lexicase Selection相比，它自动确定每个测试用例的容差（epsilon）
3. 这个算子特别适合回归问题，因为它能够：
   - 自动处理不同尺度的误差
   - 在保持选择压力的同时允许一定的误差容忍度
   - 特别适合处理有噪声的数据

举个具体例子来说明为什么要用AutomaticEpsilonLexicase：
```python
# 假设我们有这样的回归问题数据点：
测试点1: f(0.1) = 0.01      # 小尺度
测试点2: f(10) = 100        # 大尺度
测试点3: f(100) = 10000     # 更大尺度

# 普通的Lexicase Selection可能会过分注重绝对误差
# 而AutomaticEpsilonLexicase会根据数据尺度自动调整容差：
epsilon1 ≈ 0.001   # 对小尺度数据用小容差
epsilon2 ≈ 1       # 对中等尺度数据用中等容差
epsilon3 ≈ 100     # 对大尺度数据用大容差
```

所以说"对于回归问题，需要使用AutomaticEpsilonLexicase，而对于分类问题，则使用Lexicase即可"的原因是：
- 回归问题涉及连续值，需要考虑误差容差
- 分类问题是离散的，只需考虑对错，不需要容差机制

这样的设置能让算法：
1. 更好地处理不同尺度的数值问题
2. 在保持选择压力的同时有适当的容错能力
3. 避免因为数值尺度差异导致的选择偏差

In [3]:
import numpy
from deap import algorithms

# 定义统计指标
stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", numpy.mean)
mstats.register("std", numpy.std)
mstats.register("min", numpy.min)
mstats.register("max", numpy.max)

# 使用默认算法
population = toolbox.population(n=20)
hof = tools.HallOfFame(1)
pop, log  = algorithms.eaSimple(population=population,
                           toolbox=toolbox, cxpb=0.5, mutpb=0.2, ngen=20, stats=mstats, halloffame=hof, verbose=True)
print(str(hof[0]))


   	      	                        fitness                        	                      size                     
   	      	-------------------------------------------------------	-----------------------------------------------
gen	nevals	avg   	gen	max  	min        	nevals	std    	avg 	gen	max	min	nevals	std    
0  	20    	2107.6	0  	12544	0.000104102	20    	2866.19	3.75	0  	7  	2  	20    	1.29904
1  	15    	2117.9	1  	14400	0.00683013 	15    	2927.31	4.65	1  	7  	2  	15    	1.55804
2  	8     	2040.69	2  	16900	0          	8     	3034.84	5.55	2  	9  	2  	8     	2.08507
3  	14    	30541  	3  	4e+06	0          	14    	261371 	8.25	3  	13 	3  	14    	2.52735
4  	15    	528.888	4  	11881	0          	15    	1656.36	8.55	4  	15 	3  	15    	3.04097
5  	9     	946.808	5  	48400	0          	9     	3536.53	8.75	5  	16 	3  	9     	2.56661
6  	12    	8429.42	6  	1.21e+06	0          	12    	67549.6	8.5 	6  	14 	3  	12    	2.99166
7  	14    	523.787	7  	12100   	0          	14    	1684.73	9.2 	7 